# Market Basket Algorithm (MBA)

In [1]:
## Import das bibliotecas
import pandas as pd
import numpy as np
import pandasql as ps
import os
import unidecode
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Opções do pandas
pd.options.display.max_rows    = 999
pd.options.display.max_columns = 999

In [2]:
## Definição defunções
def hot_encode(x):
        return 1 if x>=1 else 0

In [3]:
path = r'C:\Github\MarketBasketAnalysis\Dataset'
dataset = pd.read_csv(path+r'\dataset.csv',sep=';')
clientes = pd.read_csv(path+r'\clientes.csv',sep=';')
grupo_cliente = pd.read_csv(path+r'\grupo_cliente.csv',sep=';')
itens = pd.read_csv(path+r'\itens.csv',sep=';')

In [4]:
dataset

,id_compra,cd_compra,id_item,id_cliente,id_grupo,dt_compra,nm_quantidade,nm_vl_item,nm_vl_total
0,0,489434,3423,0,0,2009-12-01,12,6.95,83.40
1,1,489434,2740,0,0,2009-12-01,12,6.75,81.00
2,2,489434,2742,0,0,2009-12-01,12,6.75,81.00
3,3,489434,1258,0,0,2009-12-01,48,2.10,100.80
4,4,489434,617,0,0,2009-12-01,24,1.25,30.00
...,...,...,...,...,...,...,...,...,...
417529,525456,538145,2207,4382,46,2010-12-09,12,2.95,35.40
417530,525457,538145,1478,4382,46,2010-12-09,6,2.55,15.30
417531,525458,538145,2780,4382,46,2010-12-09,6,2.95,17.70
417532,525459,538145,1656,4382,46,2010-12-09,12,1.65,19.80


In [5]:
clientes

,id_cliente,cd_cliente,cliente_nome,cliente_sexo
0,1136,12346,Mila,F
1,647,12347,Christy,F
2,613,12348,Leona,F
3,549,12349,Nina,F
4,631,12351,Dora,F
...,...,...,...,...
4378,1921,18283,Russel,M
4379,3738,18284,Ora,M
4380,1880,18285,Odessa,F
4381,1227,18286,Kaiden,M


In [6]:
grupo_cliente

,id_grupo,cd_grupo_cliente,cd_pais,tx_pais,cd_estado
0,0,0,GB,United Kingdom,"London, City of"
1,1,1,GB,United Kingdom,Birmingham
2,2,2,GB,United Kingdom,Manchester
3,3,3,GB,United Kingdom,Leeds
4,4,4,GB,United Kingdom,Newcastle upon Tyne
5,5,5,GB,United Kingdom,Leicester
6,6,6,GB,United Kingdom,Glasgow City
7,7,7,GB,United Kingdom,Liverpool
8,8,8,GB,United Kingdom,Portsmouth
9,9,9,GB,United Kingdom,Southampton


In [7]:
itens

,id_item,cd_item,tx_item
0,0,10002,INFLATABLE POLITICAL GLOBE
1,1,10080,GROOVY CACTUS INFLATABLE
2,2,10109,BENDY COLOUR PENCILS
3,3,10120,DOGGY RUBBER
4,4,10123C,HEARTS WRAPPING TAPE
...,...,...,...
4026,4026,PADS,PADS TO MATCH ALL CUSHIONS
4027,4027,POST,POSTAGE
4028,4028,SP1002,KID'S CHALKBOARD/EASEL
4029,4029,TEST001,This is a test product.


In [8]:
## Query das tabelas
query_tabelas = """
SELECT
    ds.id_compra,
    ds.cd_compra,
    it.cd_item,
    it.tx_item,
    cl.cd_cliente,
    cl.cliente_nome,
    gc.cd_grupo_cliente,
    gc.cd_pais,
    gc.tx_pais,
    gc.cd_estado,
    ds.dt_compra,
    ds.nm_quantidade,
    ds.nm_vl_item,
    ds.nm_vl_total	
FROM
    dataset ds
LEFT JOIN
    clientes cl
ON
    cl.id_cliente = ds.id_cliente
LEFT JOIN
    grupo_cliente gc
ON
    gc.id_grupo = ds.id_grupo
LEFT JOIN
    itens it
ON
    it.id_item = ds.id_item
WHERE
    gc.cd_grupo_cliente in (0,1,2,3,4,6,7,10,19,22)
"""
df = ps.sqldf(query_tabelas)
df

,id_compra,cd_compra,cd_item,tx_item,cd_cliente,cliente_nome,cd_grupo_cliente,cd_pais,tx_pais,cd_estado,dt_compra,nm_quantidade,nm_vl_item,nm_vl_total
0,0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,13085,James,0,GB,United Kingdom,"London, City of",2009-12-01,12,6.95,83.4
1,1,489434,79323P,PINK CHERRY LIGHTS,13085,James,0,GB,United Kingdom,"London, City of",2009-12-01,12,6.75,81.0
2,2,489434,79323W,WHITE CHERRY LIGHTS,13085,James,0,GB,United Kingdom,"London, City of",2009-12-01,12,6.75,81.0
3,3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",13085,James,0,GB,United Kingdom,"London, City of",2009-12-01,48,2.10,100.8
4,4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,13085,James,0,GB,United Kingdom,"London, City of",2009-12-01,24,1.25,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96507,148098,536858,22554,PLASTERS IN TIN WOODLAND ANIMALS,13520,Glenda,22,GB,United Kingdom,Wolverhampton,2010-12-03,36,1.65,59.4
96508,148099,536858,21731,RED TOADSTOOL LED NIGHT LIGHT,13520,Glenda,22,GB,United Kingdom,Wolverhampton,2010-12-03,24,1.65,39.6
96509,148100,536858,20677,PINK SPOTTY BOWL,13520,Glenda,22,GB,United Kingdom,Wolverhampton,2010-12-03,16,1.25,20.0
96510,148101,536858,20750,RED RETROSPOT MINI CASES,13520,Glenda,22,GB,United Kingdom,Wolverhampton,2010-12-03,2,7.95,15.9


In [9]:
## Tratamento dos dados
grupos_query = """
SELECT DISTINCT
    cd_grupo_cliente,
    cd_estado as tx_grupo_cliente
FROM
    df
ORDER BY
    1    
"""
grupos = ps.sqldf(grupos_query)
grupos['tx_grupo_cliente'] = grupos['tx_grupo_cliente'].replace(', City of','',regex=True).replace([" ","-"],"_",regex=True).str.lower()#.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
list_grupos = grupos['cd_grupo_cliente'].values
grupos

,cd_grupo_cliente,tx_grupo_cliente
0,0,london
1,1,birmingham
2,2,manchester
3,3,leeds
4,4,newcastle_upon_tyne
5,6,glasgow_city
6,7,liverpool
7,10,nottingham
8,19,belfast
9,22,wolverhampton


In [10]:
baskets = []
for i in list_grupos:
    bask = 'basket_' + grupos.loc[grupos['cd_grupo_cliente'] == i,'tx_grupo_cliente'].values[0]
    globals()[bask] = df[df['cd_grupo_cliente'] == i].groupby(['cd_cliente', 'tx_item'])['nm_quantidade'].sum().unstack().reset_index().fillna(0).set_index('cd_cliente').applymap(hot_encode)
    print(f" {i} - {bask}: ",globals()[bask].shape)
    baskets.append(bask)

 0 - basket_london:  (300, 3419)
 1 - basket_birmingham:  (47, 1278)
 2 - basket_manchester:  (6, 159)
 3 - basket_leeds:  (17, 479)
 4 - basket_newcastle_upon_tyne:  (15, 419)
 6 - basket_glasgow_city:  (68, 1457)
 7 - basket_liverpool:  (18, 499)
 10 - basket_nottingham:  (23, 880)
 19 - basket_belfast:  (16, 424)
 22 - basket_wolverhampton:  (14, 673)


In [11]:
frequencies = []
rules = []
for i in baskets:
    frq = 'frequent_' + i.replace('basket_','')
    rls = 'rules_' + i.replace('basket_','')
    print(f"Begin - {i.replace('basket_','')}")
    globals()[frq] = fpgrowth(globals()[i],min_support=0.2,use_colnames=True)
    globals()[rls] = association_rules(globals()[frq],metric='lift',min_threshold=1).sort_values(['confidence', 'lift'], ascending =[False, False])
    print(f"End - {i.replace('basket_','')}")
    frequencies.append(frq)
    rules.append(rls)


Begin - london
End - london
Begin - birmingham
End - birmingham
Begin - manchester
End - manchester
Begin - leeds
End - leeds
Begin - newcastle_upon_tyne
End - newcastle_upon_tyne
Begin - glasgow_city
End - glasgow_city
Begin - liverpool
End - liverpool
Begin - nottingham
End - nottingham
Begin - belfast
End - belfast
Begin - wolverhampton
End - wolverhampton


In [15]:
rules

['rules_london',
 'rules_birmingham',
 'rules_manchester',
 'rules_leeds',
 'rules_newcastle_upon_tyne',
 'rules_glasgow_city',
 'rules_liverpool',
 'rules_nottingham',
 'rules_belfast',
 'rules_wolverhampton']

In [16]:
frequencies

['frequent_london',
 'frequent_birmingham',
 'frequent_manchester',
 'frequent_leeds',
 'frequent_newcastle_upon_tyne',
 'frequent_glasgow_city',
 'frequent_liverpool',
 'frequent_nottingham',
 'frequent_belfast',
 'frequent_wolverhampton']

In [14]:
from IPython.display import display
for i in rules:
    print(i)
    display(globals()[i])

rules_london


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3,(RED HANGING HEART T-LIGHT HOLDER),(WHITE HANGING HEART T-LIGHT HOLDER),0.223333,0.413333,0.200000,0.895522,2.166586,0.107689,5.615238
1,(JUMBO BAG PINK WITH WHITE SPOTS),(JUMBO BAG RED RETROSPOT),0.233333,0.323333,0.206667,0.885714,2.739323,0.131222,5.920833
4,(60 TEATIME FAIRY CAKE CASES),(PACK OF 72 RETRO SPOT CAKE CASES),0.226667,0.290000,0.200000,0.882353,3.042596,0.134267,6.035000
5,(PACK OF 72 RETRO SPOT CAKE CASES),(60 TEATIME FAIRY CAKE CASES),0.290000,0.226667,0.200000,0.689655,3.042596,0.134267,2.491852
0,(JUMBO BAG RED RETROSPOT),(JUMBO BAG PINK WITH WHITE SPOTS),0.323333,0.233333,0.206667,0.639175,2.739323,0.131222,2.124762
2,(WHITE HANGING HEART T-LIGHT HOLDER),(RED HANGING HEART T-LIGHT HOLDER),0.413333,0.223333,0.200000,0.483871,2.166586,0.107689,1.504792


rules_birmingham


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3399,(LUNCH BAG PINK RETROSPOT),"(POSTAGE, LUNCH BAG WOODLAND)",0.212766,0.234043,0.212766,1.00000,4.272727,0.162970,inf
1862,"(CIRCUS PARADE LUNCH BOX , ROUND SNACK BOXES S...","(ROUND SNACK BOXES SET OF4 WOODLAND , SPACEBOY...",0.212766,0.255319,0.212766,1.00000,3.916667,0.158443,inf
2320,"(WOODLAND CHARLOTTE BAG, CIRCUS PARADE LUNCH B...","(RED RETROSPOT CHARLOTTE BAG, SPACEBOY LUNCH B...",0.212766,0.255319,0.212766,1.00000,3.916667,0.158443,inf
2348,"(WOODLAND CHARLOTTE BAG, LUNCH BOX WITH CUTLER...","(RED RETROSPOT CHARLOTTE BAG, SPACEBOY LUNCH B...",0.212766,0.255319,0.212766,1.00000,3.916667,0.158443,inf
3096,"(PLASTERS IN TIN CIRCUS PARADE , LUNCH BOX WIT...","(PLASTERS IN TIN WOODLAND ANIMALS, STRAWBERRY ...",0.212766,0.255319,0.212766,1.00000,3.916667,0.158443,inf
...,...,...,...,...,...,...,...,...,...
2665,(POSTAGE),"(RED TOADSTOOL LED NIGHT LIGHT, LUNCH BAG RED ...",0.787234,0.255319,0.212766,0.27027,1.058559,0.011770,1.020489
2689,(POSTAGE),"(PLASTERS IN TIN WOODLAND ANIMALS, LUNCH BAG R...",0.787234,0.255319,0.212766,0.27027,1.058559,0.011770,1.020489
2966,(POSTAGE),"(PLASTERS IN TIN CIRCUS PARADE , SPACEBOY LUNC...",0.787234,0.255319,0.212766,0.27027,1.058559,0.011770,1.020489
2982,(POSTAGE),"(PLASTERS IN TIN CIRCUS PARADE , PLASTERS IN T...",0.787234,0.255319,0.212766,0.27027,1.058559,0.011770,1.020489


rules_manchester


...

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
49,"(DOLLY GIRL LUNCH BOX, ROUND SNACK BOXES SET O...","(POSTAGE, SPACEBOY LUNCH BOX )",0.217391,0.217391,0.217391,1.000000,4.600000,0.170132,inf
52,"(POSTAGE, SPACEBOY LUNCH BOX )","(DOLLY GIRL LUNCH BOX, ROUND SNACK BOXES SET O...",0.217391,0.217391,0.217391,1.000000,4.600000,0.170132,inf
62,(ZINC METAL HEART DECORATION),(WHITE HANGING HEART T-LIGHT HOLDER),0.217391,0.217391,0.217391,1.000000,4.600000,0.170132,inf
63,(WHITE HANGING HEART T-LIGHT HOLDER),(ZINC METAL HEART DECORATION),0.217391,0.217391,0.217391,1.000000,4.600000,0.170132,inf
66,"(POSTAGE, ZINC METAL HEART DECORATION)",(WHITE HANGING HEART T-LIGHT HOLDER),0.217391,0.217391,0.217391,1.000000,4.600000,0.170132,inf
68,"(POSTAGE, WHITE HANGING HEART T-LIGHT HOLDER)",(ZINC METAL HEART DECORATION),0.217391,0.217391,0.217391,1.000000,4.600000,0.170132,inf
69,(ZINC METAL HEART DECORATION),"(POSTAGE, WHITE HANGING HEART T-LIGHT HOLDER)",0.217391,0.217391,0.217391,1.000000,4.600000,0.170132,inf
71,(WHITE HANGING HEART T-LIGHT HOLDER),"(POSTAGE, ZINC METAL HEART DECORATION)",0.217391,0.217391,0.217391,1.000000,4.600000,0.170132,inf
74,(LUNCH BAG WOODLAND),(LUNCH BAG RED SPOTTY),0.217391,0.217391,0.217391,1.000000,4.600000,0.170132,inf
75,(LUNCH BAG RED SPOTTY),(LUNCH BAG WOODLAND),0.217391,0.217391,0.217391,1.000000,4.600000,0.170132,inf


rules_nottingham
rules_belfast


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
44,(PACK OF 20 SKULL PAPER NAPKINS),(PACK OF 6 SKULL PAPER CUPS),0.2500,0.2500,0.2500,1.000000,4.000000,0.187500,inf
45,(PACK OF 6 SKULL PAPER CUPS),(PACK OF 20 SKULL PAPER NAPKINS),0.2500,0.2500,0.2500,1.000000,4.000000,0.187500,inf
46,(PACK OF 20 SKULL PAPER NAPKINS),(PACK OF 6 SKULL PAPER PLATES),0.2500,0.2500,0.2500,1.000000,4.000000,0.187500,inf
47,(PACK OF 6 SKULL PAPER PLATES),(PACK OF 20 SKULL PAPER NAPKINS),0.2500,0.2500,0.2500,1.000000,4.000000,0.187500,inf
48,(PACK OF 6 SKULL PAPER PLATES),(PACK OF 6 SKULL PAPER CUPS),0.2500,0.2500,0.2500,1.000000,4.000000,0.187500,inf
49,(PACK OF 6 SKULL PAPER CUPS),(PACK OF 6 SKULL PAPER PLATES),0.2500,0.2500,0.2500,1.000000,4.000000,0.187500,inf
50,"(PACK OF 20 SKULL PAPER NAPKINS, PACK OF 6 SKU...",(PACK OF 6 SKULL PAPER CUPS),0.2500,0.2500,0.2500,1.000000,4.000000,0.187500,inf
51,"(PACK OF 20 SKULL PAPER NAPKINS, PACK OF 6 SKU...",(PACK OF 6 SKULL PAPER PLATES),0.2500,0.2500,0.2500,1.000000,4.000000,0.187500,inf
52,"(PACK OF 6 SKULL PAPER PLATES, PACK OF 6 SKULL...",(PACK OF 20 SKULL PAPER NAPKINS),0.2500,0.2500,0.2500,1.000000,4.000000,0.187500,inf
53,(PACK OF 20 SKULL PAPER NAPKINS),"(PACK OF 6 SKULL PAPER PLATES, PACK OF 6 SKULL...",0.2500,0.2500,0.2500,1.000000,4.000000,0.187500,inf


rules_wolverhampton


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2400,"(FAWN BLUE HOT WATER BOTTLE, POSTAGE)","(BLACK/BLUE POLKADOT UMBRELLA, DOORMAT RED SPOT)",0.214286,0.214286,0.214286,1.000000,4.666667,0.168367,inf
2401,"(FAWN BLUE HOT WATER BOTTLE, BLACK/BLUE POLKAD...","(POSTAGE, DOORMAT RED SPOT)",0.214286,0.214286,0.214286,1.000000,4.666667,0.168367,inf
2404,"(POSTAGE, DOORMAT RED SPOT)","(FAWN BLUE HOT WATER BOTTLE, BLACK/BLUE POLKAD...",0.214286,0.214286,0.214286,1.000000,4.666667,0.168367,inf
2405,"(BLACK/BLUE POLKADOT UMBRELLA, DOORMAT RED SPOT)","(FAWN BLUE HOT WATER BOTTLE, POSTAGE)",0.214286,0.214286,0.214286,1.000000,4.666667,0.168367,inf
2424,(DOLLY GIRL LUNCH BOX),(CIRCUS PARADE LUNCH BOX ),0.214286,0.214286,0.214286,1.000000,4.666667,0.168367,inf
...,...,...,...,...,...,...,...,...,...
2313,(POSTAGE),(RED/WHITE DOTS RUFFLED UMBRELLA),0.642857,0.285714,0.214286,0.333333,1.166667,0.030612,1.071429
2324,(POSTAGE),(WALL TIDY RETROSPOT ),0.642857,0.285714,0.214286,0.333333,1.166667,0.030612,1.071429
21746,(POSTAGE),(RED TOADSTOOL LED NIGHT LIGHT),0.642857,0.285714,0.214286,0.333333,1.166667,0.030612,1.071429
21761,(POSTAGE),"(WOODLAND CHARLOTTE BAG, RED TOADSTOOL LED NIG...",0.642857,0.285714,0.214286,0.333333,1.166667,0.030612,1.071429
